In [1]:
import tensorflow as tf


In [8]:
folder = '../config/ppo/results/wlong_path40/3DBall/'
file = 'model-500806.ckpt.meta'
print(folder + file)

../config/ppo/results/wlong_path40/3DBall/model-500806.ckpt.meta


In [94]:
tf.compat.v1.enable_eager_execution()
import numpy as np
with tf.compat.v1.Session() as sess:
  new_saver = tf.compat.v1.train.import_meta_graph(folder+file)
  new_saver.restore(sess, tf.compat.v1.train.latest_checkpoint(folder))
  
  graph = tf.compat.v1.get_default_graph()
  op_to_restore = graph.get_operation_by_name("action")
  logdir = '/home/sai/output'
  writer = tf.summary.create_file_writer(logdir)
  rand_arr = np.random.rand(1,42)
  w1 = tf.compat.v1.placeholder_with_default(rand_arr , shape=(1,42), name="vector_observation")
  print(sess.run(op_to_restore, w1))
  tf.summary.trace_on(graph=True, profiler=True)
# Call only one tf.function when tracing.
  with writer.as_default():
    tf.summary.trace_export(
        name="my_func_trace",
        step=0,
        profiler_outdir=logdir)
  #writer.close()

INFO:tensorflow:Restoring parameters from ../config/ppo/results/wlong_path19/3DBall/model-500806.ckpt


OperatorNotAllowedInGraphError: using a `tf.Tensor` as a Python `bool` is not allowed in Graph execution. Use Eager execution or decorate this function with @tf.function.

In [55]:
import tensorboard
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [56]:
%tensorboard --logdir logdir

Reusing TensorBoard on port 6008 (pid 10591), started 0:20:14 ago. (Use '!kill 10591' to kill it.)

In [24]:
w1 = tf.compat.v1.placeholder("float", name = "vector_observation")

RuntimeError: tf.placeholder() is not compatible with eager execution.

In [59]:
tensorboard.__version__

'2.0.1'

In [61]:
current_graph = tf.compat.v1.get_default_graph()

all_names = [op.name for op in current_graph.get_operations()]

In [62]:
all_names


['global_step/initial_value',
 'global_step',
 'global_step/Assign',
 'global_step/read',
 'steps_to_increment',
 'Add',
 'Assign',
 'vector_observation',
 'normalization_steps/Initializer/zeros',
 'normalization_steps',
 'normalization_steps/Assign',
 'normalization_steps/read',
 'running_mean/Initializer/zeros',
 'running_mean',
 'running_mean/Assign',
 'running_mean/read',
 'running_variance/Initializer/ones',
 'running_variance',
 'running_variance/Assign',
 'running_variance/read',
 'Shape',
 'strided_slice/stack',
 'strided_slice/stack_1',
 'strided_slice/stack_2',
 'strided_slice',
 'Add_1',
 'Sub',
 'Cast',
 'truediv',
 'Sum/reduction_indices',
 'Sum',
 'add_2',
 'Sub_1',
 'mul',
 'Sum_1/reduction_indices',
 'Sum_1',
 'add_3',
 'Assign_1',
 'Assign_2',
 'Assign_3',
 'group_deps',
 'sub_2',
 'Cast_1',
 'add_4/y',
 'add_4',
 'truediv_1',
 'Sqrt',
 'truediv_2',
 'normalized_state/Minimum/y',
 'normalized_state/Minimum',
 'normalized_state/y',
 'normalized_state',
 'batch_size',
 '

In [1]:
import tensorflow as tf
import numpy as np
test_images = np.random.rand(1,42).astype(np.float32)

In [2]:
folder = '../config/ppo/results/wlong_path40/3DBall/'
file = 'model-500806.ckpt.meta'
print(folder + file)
from mlagents_envs.environment import UnityEnvironment

from gym_unity.envs import UnityToGymWrapper

../config/ppo/results/wlong_path40/3DBall/model-500806.ckpt.meta


In [25]:
env = UnityEnvironment(worker_id = 0)
envg = UnityToGymWrapper(env)

2020-07-31 00:10:05 INFO [environment.py:199] Listening on port 5004. Start training by pressing the Play button in the Unity Editor.
2020-07-31 00:10:51 INFO [environment.py:108] Connected to Unity environment with package version 1.0.2-preview and communication version 1.0.0


UnityTimeOutException: The Unity environment took too long to respond. Make sure that :
	 The environment does not need user interaction to launch
	 The Agents are linked to the appropriate Brains
	 The environment and the Python interface have compatible versions.

In [ ]:
folder = '../config/ppo/results/wlong_path43/3DBall/'
def wrap_frozen_graph(graph_def, inputs, outputs, print_graph=False):
    def _imports_graph_def():
        tf.compat.v1.import_graph_def(graph_def, name="")

    wrapped_import = tf.compat.v1.wrap_function(_imports_graph_def, [])
    import_graph = wrapped_import.graph

    if print_graph == True:
        print("-" * 50)
        print("Frozen model layers: ")
        layers = [op.name for op in import_graph.get_operations()]
        for layer in layers:
            print(layer)
        print("-" * 50)

    return wrapped_import.prune(
        tf.nest.map_structure(import_graph.as_graph_element, inputs),
        tf.nest.map_structure(import_graph.as_graph_element, outputs))
file = "frozen_graph_def.pb"
with tf.io.gfile.GFile(folder + file, "rb") as f:
        graph_def = tf.compat.v1.GraphDef()
        loaded = graph_def.ParseFromString(f.read())
frozen_func = wrap_frozen_graph(graph_def=graph_def,inputs=["vector_observation:0"],outputs=["action:0"],print_graph=True)
print("-" * 50)
print("Frozen model inputs: ")
print(frozen_func.inputs)
print("Frozen model outputs: ")
print(frozen_func.outputs)

    # Get predictions for test imag

frozen_graph_predictions = frozen_func(vector_observation=tf.constant(test_images))[0]

    # Print the prediction for the first image
print("-" * 50)
print("Example TensorFlow frozen graph prediction reference:")
print(frozen_graph_predictions[0].numpy())
import time
t0 = time.time()
observation = envg.reset()
for i in range(0,1000):
    
    frozen_func = wrap_frozen_graph(graph_def=graph_def,inputs=["vector_observation:0"],outputs=["action:0"],print_graph=False)
    frozen_graph_predictions = frozen_func(vector_observation=tf.constant(observation.reshape(1,42)))[0]
    #frozen_graph_predictions = frozen_func(vector_observation=tf.constant(test_images))[0]
    #print(frozen_graph_predictions[0].numpy())
    observation, reward, done, info = envg.step(frozen_graph_predictions[0].numpy())
t1 = time.time()
print(t1-t0)
#frozen_graph_predictions = frozen_func(vector_observation=tf.constant(test_images))[0]
#print(frozen_graph_predictions[0].numpy())

    # The two predictions should be almost the same.
    #assert np.allclose(a=frozen_graph_predictions[0].numpy(), b=predictions[0], rtol=1e-05, atol=1e-08, equal_nan=False)

In [17]:
observation = envg.reset()

In [6]:
observation

array([ 2.8856206e-01,  5.5187140e+00,  5.9175491e-04,  2.8245471e+00,
        4.0332171e-01,  3.2743881e+00, -7.7486038e-04,  2.9465220e+00,
        9.0855438e-01,  3.0654228e+00,  4.1561127e-03,  2.9378021e+00,
        1.4213042e+00,  3.0087471e+00,  1.0493994e-02,  2.9350741e+00,
        1.9461168e+00,  2.9835088e+00,  2.7450085e-02,  2.9537461e+00,
        2.4504504e+00,  2.9729600e+00,  4.3011427e-02,  2.9474940e+00,
        2.9807472e+00,  2.9684918e+00,  4.6428442e-02,  2.9387300e+00,
        3.4861815e+00,  2.9659939e+00,  4.9986839e-02,  2.9418941e+00,
        3.9913073e+00,  2.9645801e+00,  5.4682493e-02,  2.9621961e+00,
        4.5108919e+00,  2.9639971e+00,  5.6025267e-02,  2.9671619e+00,
        1.6021711e+00, -1.4439966e-01], dtype=float32)

In [22]:
for i in range(0,100):
    observation = envg.step([0., 0.])

In [23]:
observation

(array([ 3.1718987e-01,  2.6397274e+00, -7.1165800e-02,  2.9253261e+00,
         8.2946199e-01,  2.4810174e+00, -2.3757100e-01,  2.9868939e+00,
         1.4194126e+00,  2.3704438e+00, -3.9009976e-01,  2.9857869e+00,
         2.0244434e+00,  2.2775273e+00, -4.8208833e-01,  3.2309980e+00,
         2.5908678e+00,  2.2095571e+00, -5.5408025e-01,  3.0272779e+00,
         3.1385303e+00,  2.1544523e+00, -5.6134295e-01,  2.8097680e+00,
         3.6512144e+00,  2.1172132e+00, -5.9754920e-01,  2.6893539e+00,
         4.2099533e+00,  2.0821514e+00, -6.1818290e-01,  2.6712611e+00,
         4.7472596e+00,  2.0539546e+00, -6.3812470e-01,  2.9293561e+00,
         5.2431216e+00,  2.0312028e+00, -6.4151406e-01,  2.9405370e+00,
         3.7776546e-03, -1.4553810e-03], dtype=float32),
 0.0,
 False,
 {'step': <mlagents_envs.base_env.DecisionSteps at 0x7f034828b1d0>})

In [26]:
envg.close()

UnityEnvironmentException: No Unity environment is loaded.